# A simple LLT extension to LDVM to account for finite (large) aspect ratios

In [24]:
workspace()
include("../src/UNSflow.jl")
using UNSflow


    /(Real, ForwardDiff.GradientNumber) at /Users/kiran/.julia/v0.4/ForwardDiff/src/GradientNumber.jl:98
is ambiguous with: 
    /(Main.ForwardDiff.GradientNumber, Real) at /Users/kiran/.julia/v0.4/ForwardDiff/src/GradientNumber.jl:105.
To fix, define 
    /(Main.ForwardDiff.GradientNumber, ForwardDiff.GradientNumber)
before the new definition.
    /(Real, ForwardDiff.GradientNumber) at /Users/kiran/.julia/v0.4/ForwardDiff/src/GradientNumber.jl:98
is ambiguous with: 
    /(Main.ForwardDiff.HessianNumber, Real) at /Users/kiran/.julia/v0.4/ForwardDiff/src/HessianNumber.jl:147.
To fix, define 
    /(Main.ForwardDiff.HessianNumber, ForwardDiff.GradientNumber)
before the new definition.
    /(Real, ForwardDiff.GradientNumber) at /Users/kiran/.julia/v0.4/ForwardDiff/src/GradientNumber.jl:98
is ambiguous with: 
    /(Main.ForwardDiff.TensorNumber, Real) at /Users/kiran/.julia/v0.4/ForwardDiff/src/TensorNumber.jl:175.
To fix, define 
    /(Main.ForwardDiff.TensorNumber, ForwardDiff.GradientNum

In [27]:
alphadef = EldUpDef(45*pi/180,pi/8,0.9)
hdef = ConstDef(0.)
udef = ConstDef(1.)
full_kinem = KinemDef(alphadef, hdef, udef)

pvt = 0.0 #leading edge

surf = TwoDSurf(1., 1., "FlatPlate", pvt, 70, 35, "Prescribed", full_kinem)

curfield = TwoDFlowField()

nsteps =round(Int,2.5/0.015)+1

mat, surf, curfield = ldvm(surf, curfield, nsteps)

(
168x9 Array{Float64,2}:
 0.015  -5.55112e-17  0.0  1.0  4.22877e-6  …  -4.44408e-7    5.68583e-6
 0.03    5.55112e-17  0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.045   0.0          0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.06   -5.55112e-17  0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.075  -5.55112e-17  0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.09    5.55112e-17  0.0  1.0  4.22877e-6  …   0.000343588   5.68583e-6
 0.105   0.0          0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.12    5.55112e-17  0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.135   5.55112e-17  0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.15    0.0          0.0  1.0  4.22877e-6      0.000343588   5.68583e-6
 0.165  -5.55112e-17  0.0  1.0  4.22877e-6  …   0.000343588   5.68584e-6
 0.18    5.55112e-17  0.0  1.0  4.22877e-6      0.000343588   5.68584e-6
 0.195   0.0          0.0  1.0  4.22877e-6      0.000343588   5.68584e-6
 ⋮                       

In [31]:
using PyPlot
pygui(true)
plot(mat[:,1],mat[:,6])

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x32f511250>

In [7]:
n_step = length(bnd_circ2d)

525

In [8]:
c = 1

1

In [9]:
s = 4 #semispan

4

In [10]:
u = 1

1

In [11]:
n_bterm = 20
n_span = 10

10

In [12]:
lhs = zeros(n_span,n_bterm)
rhs = zeros(n_span)
b_coeff = zeros(n_bterm)
bcoeff_prev = zeros(n_bterm)
bdot = zeros(n_bterm)
sp_gam = zeros(nsteps,n_span)
psi = zeros(n_span)
dt = mat[2,1]-mat[1,1]
cnc_f = zeros(nsteps)
cnnc_f = zeros(nsteps)

for i = 1:n_span
    psi[i] = (real(i)/(n_span+1))*pi
end

for i = 1:nsteps
    for j = 1:n_span
        for n = 1:n_bterm
            lhs[j,n] = sin(n*psi[j])*(sin(psi[j]) + (n*c*pi/(4*s)))
        end
        rhs[j] = c*pi*sin(psi[j])*bnd_circ2d[i]/(4*s)
    end
    bcoeff_prev = b_coeff
    b_coeff = \(lhs, rhs)
    bdot = (b_coeff - bcoeff_prev)/dt    
    for j = 1:n_span
        sp_gam[i,j] = 0
        for n = 1:n_bterm
            sp_gam[i,j] = sp_gam[i,j] + 4*s*u*b_coeff[n]*sin(n*psi[j])
        end
    end
    sum_bcoeff = 0
    for n = 1:n_bterm
        if rem(n,2) != 0 
            sum_bcoeff = sum_bcoeff + b_coeff[n]
        end
    end
    cnc_f[i] = -2*pi*(surf.kinem.u*cos(surf.kinem.alpha)/surf.uref + surf.kinem.hdot*sin(surf.kinem.alpha)/surf.uref)*(sum_bcoeff)
    sum_bdot = 0
    for n = 1:n_bterm
        if rem(n,2) != 0 
            sum_bdot = sum_bdot + bdot[n]
        end
    end 
    cnnc_f[i] = -(2*pi*surf.c/(surf.uref))*(3*sum_bdot/4)
end

cl_2d = mat[:,6]
cl_3d = mat[:,6] + cnc_f 

525-element Array{Float64,1}:
 -393.138  
   -3.31613
   -6.13143
   -8.24899
   -8.92473
   -9.05422
   -8.76605
   -8.28266
   -7.82295
   -7.46914
   -7.24301
   -7.12122
   -7.06081
    ⋮      
   -7.42717
   -7.40378
   -7.32991
   -7.27373
   -7.26863
   -7.36227
   -7.3566 
   -7.26515
   -7.27242
   -7.27939
   -7.27388
   -7.27535

In [13]:
using PyPlot


In [22]:
pygui(true)
axis([0, 1.8, -30, 30])
plot(mat[:,1],cl_3d)
plot(mat[:,1],cl_2d)

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x3231220d0>

In [23]:
typeof(mat)

Array{Float64,2}